<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
!pip install ray

In [1]:
import time
import numpy as np
from cartesian_explorer import Explorer
from cartesian_explorer import parallels
from cartesian_explorer import caches 

In [2]:
def payload(rnd):
    N = 2000
    product = 1
    for j in range(N):
        product *= np.sin(j/N)**2 + np.cos(j/N)**2
    return product

In [3]:
def test_backend(parallel, cache):
    ex = Explorer(parallel=parallel, cache=cache)
    _payload = ex.provider(cache=True)(payload)


    start = time.time()
    %timeit ex.map_variable('payload', rnd=np.random.randn(8))
    end = time.time()
    print(f'Duration = {end-start}')
   

In [4]:
def test_with_caches(Parallel):
    for Cache in [caches.JobLibCache, caches.FunctoolsCache_Disk]:
        cache = Cache('/tmp/test_caex')
        print('\n-cache=', cache)
        try:
            for nproc in [1, 2]:
                print('-- nproc=', nproc)
                parallel = Parallel(processes=nproc)
                test_backend(parallel, cache)
        except Exception as e:
            print('!Failed to distribute tasks:', e)


In [5]:
test_with_caches(parallels.Ray)


-cache= <cartesian_explorer.caches.JobLibCache object at 0x7fb49849bcd0>
-- nproc= 1


2020-11-19 15:25:18,861	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-19 15:25:24,340	ERROR worker.py:660 -- Calling ray.init() again after it has already been called.


76.4 ms ± 6.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Duration = 2.4282917976379395
-- nproc= 2


2020-11-19 15:25:25,062	ERROR worker.py:660 -- Calling ray.init() again after it has already been called.


72.8 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Duration = 0.7205488681793213

-cache= <cartesian_explorer.caches.FunctoolsCache_Disk object at 0x7fb447d7c220>
-- nproc= 1
!Failed to distribute tasks: cannot pickle '_thread.RLock' object


In [6]:
test_with_caches(parallels.Thread)


-cache= <cartesian_explorer.caches.JobLibCache object at 0x7fb49849bcd0>
-- nproc= 1
51.6 ms ± 3.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Duration = 4.299301862716675
-- nproc= 2
52.6 ms ± 2.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Duration = 4.2304534912109375

-cache= <cartesian_explorer.caches.FunctoolsCache_Disk object at 0x7fb44550fe20>
-- nproc= 1
92.7 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Duration = 7.207552909851074
-- nproc= 2
69.7 ms ± 7.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Duration = 5.636850833892822


In [8]:
test_with_caches(parallels.Multiprocess)


-cache= <cartesian_explorer.caches.JobLibCache object at 0x7fb444162220>
-- nproc= 1
144 ms ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Duration = 11.103675127029419
-- nproc= 2
159 ms ± 59.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Duration = 1.3521721363067627

-cache= <cartesian_explorer.caches.FunctoolsCache_Disk object at 0x7fb4984a6a30>
-- nproc= 1
!Failed to distribute tasks: Can't pickle <function payload at 0x7fb445588670>: it's not the same object as __main__.payload


In [11]:
try:
    parallel = parallels.Thread(processes=1)
    test_backend(parallel)
    parallel = parallels.Thread(processes=2)
    test_backend(parallel)
except Exception as e:
    print('Failed to distribute tasks:', e)

999.9999999999957
Duration = 10.58320426940918
999.9999999999957
Duration = 11.23127555847168


In [13]:
try:
    parallel = parallels.Multiprocess(processes=1)
    test_backend(parallel)
    parallel = parallels.Multiprocess(processes=2)
    test_backend(parallel)
except Exception as e:
    print('Failed to distribute tasks:', e)

999.9999999999957
Duration = 8.987144708633423
999.9999999999957
Duration = 4.482992887496948
